# 📌 TextRank 주요 키워드 추출

In [ ]:
# 단어 그래프 생성

from collections import Counter

def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx


# sents : list of str 형식의 문장들
# tokenize : 토크나이저
# min_count : 주어진 문서 집합에서 최소 빈도수 이상 등장한 단어들

In [ ]:
# 두 단어 간의 유사도 정의를 위해 co-occurrence 계산

from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)


# window : 문장 내 두 단어의 간격이 window 인 횟수 (2-8 사이의 값 추천)
# min_cooccurrence ; 그래프가 지나치게 dense 해지는 것을 방지, 그래프를 sparse 하게 만들 수 있음

In [ ]:
# dict_to_mat : dict of dict 형식의 그래프를 spicy의 sparse matrix(희소행렬)로 변환

from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

In [ ]:
# word_graph : 단어 그래프를 만드는데 명사, 동사, 형용사와 같은 단어만 사용

def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence) # verbose 제거
    return g, idx_to_vocab

In [ ]:
# 만들어진 그래프에 PageRank 학습하는 함수 생성

import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

In [ ]:
# TextRank Keyword Extraction
# 상위 30개 키워드 추출로 설정

def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

### ✔️ Tokenizer 선언 및 비교
##### - 선언 : 명사만 추출하도록 정의함

In [ ]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

In [ ]:
# Tokenizer = Komoran

komoran = Komoran()

def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w)]
    return words

In [ ]:
# Tokenizer = Kkma

kkma = Kkma()

def kkma_tokenizer(sent):
    words = kkma.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w)]
    return words

In [ ]:
# Tokenizer = Hannanum

hannanum = Hannanum()

def hannanum_tokenizer(sent):
    words = hannanum.pos(sent, join=True)
    words = [w for w in words if ('N' in w)]
    return words

In [ ]:
# Tokenizer = Okt

okt = Okt()

def okt_tokenizer(sent):
    words = okt.pos(sent, join=True)
    words = [w for w in words if ('Noun' in w)]
    return words

In [ ]:
# Tokenizer = Mecab

mecab = Mecab()

def mecab_tokenizer(sent):
    words = mecab.pos(sent, join=True)
    words = [w for w in words if ('NNG' in w or 'NNP' in w)]
    return words

##### - 비교 : Tokenizer에 따른 결과 비교

In [ ]:
sample = """
국내 대학 최초 기상기후데이터 융합분석 특성화대학원 대학원생 장학금 혜택 및 채용 연계형 인턴십 기회도 제공 엘텍공과대학 기후·에너지시스템공학과(학과장 허진)가 국내 대학 최초로 기상기후데이터 융합분석 특성화대학원으로 선정됐다. (앞줄 왼쪽부터) 박선기 교수, 이정민 한국기상산업기술원 본부장, 이향숙 산학협력단장, 김은미 총장, 유희동 기상청 차장, 안영인 한국기상산업기술원장, 이정환 기상청 국장 (뒷줄 왼쪽부터) 최용상 교수, 민배현 교수, 안명환 교수, 허진 교수, 박동일 한국기상산업기술원 실장 기상기후데이터 융합분석 특성화대학원 사업은 기상청과 한국기상산업기술원이 기상기후 빅데이터에 기반한 미래 신산업을 이끌 전문인재 양성을 위해 올해 처음 도입한 사업이다. 기후변화로 인한 기상재해 규모가 날로 커지는 상황에서 기상기후 빅데이터 분석·활용능력과 기후변화 대응능력은 미래 국가경쟁력에 직접적 영향을 주는 요소로 중요성이 매우 커지고 있다. 이와 관련해 본교와 한국기상산업기술원은 6월 10일(금) ECC에서 ‘기상기후데이터 융합분석 특성화대학원 사업 협약식’을 개최하고, 기상기후전문가 양성에 협력해 가기로 했다. 이날 협약식에는 본교 이향숙 산학협력단장, 기후·에너지시스템공학과 교수진, 그리고 이정환 기상청 기상서비스진흥국장, 안영인 한국기상산업기술원장을 비롯한 기상청 및 기술원 관계자들이 참석했으며, 협약식 후 김은미 총장과 유희동 기상청 차장이 자리해 인재양성 방안에 대한 논의를 진행했다. 이번 특성화대학원 사업 선정으로 본교는 향후 4년 7개월간 정부지원금과 대응자금을 합쳐 총 22억 6천 8백만 원의 사업비를 확보하게 됐다. 오는 2학기부터 기상기후데이터 융합·분석·활용 관련 교과목을 이수하고 산학협력 프로젝트에 참여하는 대학원생을 대상으로 장학생을 선발한다. (왼쪽) 이향숙 산학협력단장과 안영인 한국기상산업기술원장 본교는 기상기후데이터 융합분석 특성화대학원의 석·박사과정 신입생과 트랙이수과정 학생을 연간 15명 이상 모집하고, 올해 9월부터 대학원을 운영하여 기상기후데이터 융합·분석·활용 분야 전문 지식을 갖춘 석·박사급 인력을 양성할 계획이다. 인공지능 기법을 활용한 기상예측, 기후모델링, 신재생 및 수소에너지 데이터 분석 등 다양한 교과목이 개설되며, 채용 연계형 기업 인턴십 프로그램도 활성화 된다. 또한 매년 기상기후 및 다양한 융복합 분야의 지속적 혁신과 성장을 이끌 수 있는 전문인력을 배출하고 국제학술논문을 발표할 계획이다. 특성화대학원 교육과정을 이수한 인재들은 기상기후데이터 융합·분석·활용 분야 등 산업현장에 바로 투입할 수 있는 역량을 갖추게 된다. 본 사업의 책임자인 유창현 교수는 “학과 교수님 모두 합심해 이뤄낸 성과로서 본교에서 고도의 전문성을 갖는 기상기후데이터 분석 전문인력이 선도적으로 배출될 것이라 기대된다”고 사업 선정의 소감을 밝혔다. 또한 학과에서 지속가능경영 과목을 담당하고 있는 황재학 겸임교수(금융감독원 선임조사역)는 “기업 ESG대응팀 등이 기업의 기후리스크 관리를 위해 기상기후데이터 전문인력에 대한 선호가 급증할 것“이라며 이화여대 특성화대학원 출신의 석박사 학생들이 산업계 전반에 매우 수월하게 진출하게 될 것이라고 전망했다. 기후·에너지시스템공학과는 현재 한국전력, 환경부, 기상청 등의 공공기관 및 민간 기업들의 다양한 지원을 받고 있으며, 지난해 학과 교수진 전원이 참여해 국내 주요 기업들(SK이노베이션, 삼성바이오로직스, CJ, 신한금융, KB금융)을 대상으로 ESG 투자 및 공시 규제를 대비한 기후리스크 관리 모형을 개발했다. 올해 6월부터는 신규 기업들(교보생명, NH금융지주, 하나금융지주, 한화투자증권, 한화솔루션, 한화토탈)을 대상으로 개발을 시작한다. 2016년 신설 이래 다양한 연구와 사업을 수행하며 빠르게 성장하고 있는 기후·에너지시스템공학과는 미래지향적 융합 교육과 연구로 기후대응·에너지·ESG경영 등의 복합적 이슈를 풀어갈 수 있는 국내 대표 학과로 자리 매김하고 있다.
"""

In [ ]:
komoran_tokenizer(sample)

In [ ]:
kkma_tokenizer(sample)

In [ ]:
hannanum_tokenizer(sample)

In [ ]:
okt_tokenizer(sample)

In [ ]:
mecab_tokenizer(sample)

### ✔️ Sample Test : textrank_keyword 사용해보기

In [ ]:
sample_list = ["""
국내 대학 최초 기상기후데이터 융합분석 특성화대학원 대학원생 장학금 혜택 및 채용 연계형 인턴십 기회도 제공 엘텍공과대학 기후·에너지시스템공학과(학과장 허진)가 국내 대학 최초로 기상기후데이터 융합분석 특성화대학원으로 선정됐다. (앞줄 왼쪽부터) 박선기 교수, 이정민 한국기상산업기술원 본부장, 이향숙 산학협력단장, 김은미 총장, 유희동 기상청 차장, 안영인 한국기상산업기술원장, 이정환 기상청 국장 (뒷줄 왼쪽부터) 최용상 교수, 민배현 교수, 안명환 교수, 허진 교수, 박동일 한국기상산업기술원 실장 기상기후데이터 융합분석 특성화대학원 사업은 기상청과 한국기상산업기술원이 기상기후 빅데이터에 기반한 미래 신산업을 이끌 전문인재 양성을 위해 올해 처음 도입한 사업이다. 기후변화로 인한 기상재해 규모가 날로 커지는 상황에서 기상기후 빅데이터 분석·활용능력과 기후변화 대응능력은 미래 국가경쟁력에 직접적 영향을 주는 요소로 중요성이 매우 커지고 있다. 이와 관련해 본교와 한국기상산업기술원은 6월 10일(금) ECC에서 ‘기상기후데이터 융합분석 특성화대학원 사업 협약식’을 개최하고, 기상기후전문가 양성에 협력해 가기로 했다. 이날 협약식에는 본교 이향숙 산학협력단장, 기후·에너지시스템공학과 교수진, 그리고 이정환 기상청 기상서비스진흥국장, 안영인 한국기상산업기술원장을 비롯한 기상청 및 기술원 관계자들이 참석했으며, 협약식 후 김은미 총장과 유희동 기상청 차장이 자리해 인재양성 방안에 대한 논의를 진행했다. 이번 특성화대학원 사업 선정으로 본교는 향후 4년 7개월간 정부지원금과 대응자금을 합쳐 총 22억 6천 8백만 원의 사업비를 확보하게 됐다. 오는 2학기부터 기상기후데이터 융합·분석·활용 관련 교과목을 이수하고 산학협력 프로젝트에 참여하는 대학원생을 대상으로 장학생을 선발한다. (왼쪽) 이향숙 산학협력단장과 안영인 한국기상산업기술원장 본교는 기상기후데이터 융합분석 특성화대학원의 석·박사과정 신입생과 트랙이수과정 학생을 연간 15명 이상 모집하고, 올해 9월부터 대학원을 운영하여 기상기후데이터 융합·분석·활용 분야 전문 지식을 갖춘 석·박사급 인력을 양성할 계획이다. 인공지능 기법을 활용한 기상예측, 기후모델링, 신재생 및 수소에너지 데이터 분석 등 다양한 교과목이 개설되며, 채용 연계형 기업 인턴십 프로그램도 활성화 된다. 또한 매년 기상기후 및 다양한 융복합 분야의 지속적 혁신과 성장을 이끌 수 있는 전문인력을 배출하고 국제학술논문을 발표할 계획이다. 특성화대학원 교육과정을 이수한 인재들은 기상기후데이터 융합·분석·활용 분야 등 산업현장에 바로 투입할 수 있는 역량을 갖추게 된다. 본 사업의 책임자인 유창현 교수는 “학과 교수님 모두 합심해 이뤄낸 성과로서 본교에서 고도의 전문성을 갖는 기상기후데이터 분석 전문인력이 선도적으로 배출될 것이라 기대된다”고 사업 선정의 소감을 밝혔다. 또한 학과에서 지속가능경영 과목을 담당하고 있는 황재학 겸임교수(금융감독원 선임조사역)는 “기업 ESG대응팀 등이 기업의 기후리스크 관리를 위해 기상기후데이터 전문인력에 대한 선호가 급증할 것“이라며 이화여대 특성화대학원 출신의 석박사 학생들이 산업계 전반에 매우 수월하게 진출하게 될 것이라고 전망했다. 기후·에너지시스템공학과는 현재 한국전력, 환경부, 기상청 등의 공공기관 및 민간 기업들의 다양한 지원을 받고 있으며, 지난해 학과 교수진 전원이 참여해 국내 주요 기업들(SK이노베이션, 삼성바이오로직스, CJ, 신한금융, KB금융)을 대상으로 ESG 투자 및 공시 규제를 대비한 기후리스크 관리 모형을 개발했다. 올해 6월부터는 신규 기업들(교보생명, NH금융지주, 하나금융지주, 한화투자증권, 한화솔루션, 한화토탈)을 대상으로 개발을 시작한다. 2016년 신설 이래 다양한 연구와 사업을 수행하며 빠르게 성장하고 있는 기후·에너지시스템공학과는 미래지향적 융합 교육과 연구로 기후대응·에너지·ESG경영 등의 복합적 이슈를 풀어갈 수 있는 국내 대표 학과로 자리 매김하고 있다.
"""]

In [ ]:
# Textrank로부터 계산된 랭킹값 : 단어 간 상대적인 중요도
# tokenizer = mecab
# textrank_keyword(list of str 형식으로 넣어주기)

textrank_keyword(sample_list, tokenize=mecab_tokenizer, min_count=2, window=2, min_cooccurrence=1)

# 📌 df2kwdf 함수생성

> **Dataframe to Keyword Dataframe**
> - parameter : df (원데이터프레임), category_num (5분류)
> - argument : 데이터프레임, 1-5 사이의 숫자를 인자로 받음
> - tokenizer : Mecab
> - return : 키워드가 추출된 데이터프레임 반환 (columns : 키워드, 품사, 랭킹값)

> **사용방법**
> - df2kwdf(df, 1)
> - ex. df = 경북대학교 홈페이지 뉴스기사 데이터.csv
> - 경북대학교 1번 분류에 대한 textrank에 기반한 주요 키워드 추출

In [ ]:
def df2kwdf(df, category_num):
    class_ = pd.DataFrame(df[df['5p']==category_num]) # 데이터프레임에 따라 칼럼명 변경하기
    class_list = [text for text in class_['내용_전처리']] # 데이터프레임에 따라 칼럼명 변경하기
    
    class_mecab = list(textrank_keyword(class_list, tokenize=mecab_tokenizer, min_count=2, window=2, min_cooccurrence=1))
    
    kw_list = []
    
    for i in range(len(class_mecab)):
        kw_str = class_mecab[i][0].split('/')[0]
        kw_pos = class_mecab[i][0].split('/')[1]
        kw_rank = class_mecab[i][1]
        list_ = [kw_str, kw_pos, kw_rank]
        kw_list.append(list_)

    kwdf = pd.DataFrame(kw_list, columns=['키워드', '품사', 'textrank'])

    return kwdf

### ✔️ Sample Test : 경북대학교 데이터로 df2kwdf 사용해보기

In [ ]:
import pandas as pd

df = pd.read_csv('/Users/jieunlee/Downloads/경북5p.csv')

In [ ]:
df.info()

In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# 경북대학교 1번분류 키워드 데이터프레임 확인
df2kwdf(df, 1)

In [ ]:
# 한 번에 5분류별 주요 키워드 csv 파일로 내보내기
for i in range(1, 6):
    kwdf = df2kwdf(df, i)
    kwdf.to_csv(f'/Users/jieunlee/Downloads/국내개별대학_키워드추출_textrank/중복미제거/경북_kw{i}.csv')

##### - 확인 : 함수 미사용

In [ ]:
kb1 = pd.DataFrame(kb[kb['5p']==1])
kb2 = pd.DataFrame(kb[kb['5p']==2])
kb3 = pd.DataFrame(kb[kb['5p']==3])
kb4 = pd.DataFrame(kb[kb['5p']==4])
kb5 = pd.DataFrame(kb[kb['5p']==5])

In [ ]:
kb1_list = [text for text in kb1['내용_전처리']]
kb2_list = [text for text in kb1['내용_전처리']]
kb3_list = [text for text in kb1['내용_전처리']]
kb4_list = [text for text in kb1['내용_전처리']]
kb5_list = [text for text in kb1['내용_전처리']]

In [ ]:
textrank_keyword(kb1_list, tokenize=mecab_tokenizer, min_count=2, window=2, min_cooccurrence=1)

In [ ]:
textrank_keyword(kb2_list, tokenize=mecab_tokenizer, min_count=2, window=2, min_cooccurrence=1)

In [ ]:
textrank_keyword(kb3_list, tokenize=mecab_tokenizer, min_count=2, window=2, min_cooccurrence=1)

In [ ]:
textrank_keyword(kb4_list, tokenize=mecab_tokenizer, min_count=2, window=2, min_cooccurrence=1)

In [ ]:
textrank_keyword(kb5_list, tokenize=mecab_tokenizer, min_count=2, window=2, min_cooccurrence=1)

# 📌 5분류별 고유 키워드 추출
##### - 중복 등장 키워드 제거

In [ ]:
## 해당 경로에 있는 .csv 파일명 리스트 가져오기

import os

path = '/Users/jieunlee/Downloads/국내개별대학_키워드추출_textrank/중복미제거/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝이 .csv인 경우

In [ ]:
file_list_py

In [ ]:
# csv 파일들을 각 DataFrame으로 불러오기 : data1 - data5

for i in file_list_py:
    globals()['data'+i.split('.')[0][-1]] = pd.read_csv(path + i)

In [ ]:
# 모든 키워드를 하나의 리스트에 넣기

kw_list_all = []
data_list = [data1, data2, data3, data4, data5]

for data in data_list:
    for kw in data['키워드']:
        kw_list_all.append(kw)

In [ ]:
data_list # 30*5 = 150개 데이터

In [ ]:
len(kw_list_all)

In [ ]:
# 모든 키워드 리스트 내에서 중복해서 등장한 키워드 추출하기

elem = [] # 처음 등장한 값인지 판별하는 리스트
dup = [] # 중복된 원소만 넣는 리스트

for i in kw_list_all:
    if i not in elem: # 처음 등장한 원소
        elem.append(i)
    else:
        if i not in dup: # 이미 중복 원소로 판정된 경우는 제외
            dup.append(i)

print(dup) # 2회 이상 등장한 값들만 담긴 리스트

In [ ]:
# 5분류별 고유 키워드 추출 및 csv 파일로 내보내기

for kw in dup:
    for idx, data in enumerate(data_list):
        indexNames = data[data['키워드']==kw].index
        data.drop(indexNames, inplace=True)
        data.to_csv(f'/Users/jieunlee/Downloads/국내개별대학_키워드추출_textrank/중복제거/경북_kw{idx+1}.csv')